In [20]:
! pip install tweepy
! pip install numpy
! pip install pandas
! pip install torch

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [56]:
! pip install detoxify

  Using cached detoxify-0.2.2-py3-none-any.whl (11 kB)
     |████████████████████████████████| 1.2 MB 410 kB/s eta 0:00:01
     |████████████████████████████████| 804.1 MB 2.4 kB/s eta 0:00:01    |█                               | 22.7 MB 1.7 MB/s eta 0:07:38     |██▎                             | 56.1 MB 2.1 MB/s eta 0:05:57     |███▏                            | 78.4 MB 2.1 MB/s eta 0:05:51     |████████▊                       | 220.1 MB 2.5 MB/s eta 0:03:53     |██████████                      | 249.1 MB 2.5 MB/s eta 0:03:39     |███████████▉                    | 297.1 MB 2.4 MB/s eta 0:03:31     |█████████████                   | 326.2 MB 2.0 MB/s eta 0:03:58     |█████████████▊                  | 343.8 MB 2.6 MB/s eta 0:02:59     |███████████████▍                | 386.7 MB 2.5 MB/s eta 0:02:45     |████████████████▍               | 411.5 MB 1.9 MB/s eta 0:03:23     |███████████████████▎            | 485.7 MB 2.3 MB/s eta 0:02:19     |█████████████████████▏          | 531.5 MB 1.8 

In [1]:
import os
import tweepy
import numpy as np
import pandas as pd

In [2]:
TWITTER_API_KEY = os.environ.get('TWITTER_API_KEY')
TWITTER_API_SECRET = os.environ.get('TWITTER_API_SECRET')

In [3]:
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET)

In [4]:
api = tweepy.API(auth)

In [5]:
tp = api.get_user('franciscojarceo')

In [6]:
tp._json['name']

'Francisco Javier Arceo'

In [7]:
def get_all_tweets(screen_name):
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200, max_id=oldest)
        #save most recent tweets
        alltweets.extend(new_tweets)
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print(f"...{len(alltweets)} tweets downloaded so far")

    return alltweets

In [8]:
mytweets = get_all_tweets('franciscojarceo')

getting tweets before 1367851989896470533
...396 tweets downloaded so far
getting tweets before 1364683264481353727
...595 tweets downloaded so far
getting tweets before 1360787085041545215
...795 tweets downloaded so far
getting tweets before 1358427426414612488
...992 tweets downloaded so far
getting tweets before 1355285391742562308
...1192 tweets downloaded so far
getting tweets before 1353880148903395329
...1391 tweets downloaded so far
getting tweets before 1352114114114564097
...1590 tweets downloaded so far
getting tweets before 1350054769235292160
...1790 tweets downloaded so far
getting tweets before 1348990717209833471
...1987 tweets downloaded so far
getting tweets before 1346956415836033023
...2187 tweets downloaded so far
getting tweets before 1344134526654685183
...2387 tweets downloaded so far
getting tweets before 1341746018077925381
...2587 tweets downloaded so far
getting tweets before 1340050444417028095
...2787 tweets downloaded so far
getting tweets before 1337582

In [9]:
mytweets[2]._json['in_reply_to_status_id']

In [10]:
mytweets[1]._json['in_reply_to_status_id']

1375934272339591171

In [11]:
mytweets[1]._json['id'] #['text']

1375935305891602439

In [12]:
'in_reply_to_status_id' in mytweets[1]._json

True

In [13]:
res = []
for i in mytweets:
    if i._json['in_reply_to_status_id'] is not None:
        res.append((i._json['id'], i._json['text']))

In [14]:
res[0:4]

[(1375935305891602439,
  '@jkwade @polina_marinova @anafabrega11 what was the lesson? 😂'),
 (1375812451879546883, "@paul__132 bro i'm surprised she keeps me 😂😂😂"),
 (1375443648125485060, '@AtJuanMedina Move to South Dakota bro'),
 (1375277503007318020, 'i ate 2 sleeves 😂😂😂😭😭😭')]

In [15]:
from detoxify import Detoxify

In [21]:
import torch

In [ ]:
torch.hub.download_url_to_file('')

In [22]:
model = torch.hub.load('unitaryai/detoxify','toxic_bert', source='github')

Using cache found in /root/.cache/torch/hub/unitaryai_detoxify_master


ImportError: cannot import name 'toxic_albert' from 'detoxify' (/usr/local/lib/python3.7/site-packages/detoxify/__init__.py)

(1375277503007318020, 'i ate 2 sleeves 😂😂😂😭😭😭')

In [59]:
Detoxify('original').predict(res[0])

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html